<a href="https://colab.research.google.com/github/NancyYiWang/WildFireSmokePrediction/blob/main/PredictModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install rasterio

import os
import numpy as np
import xarray as xr
import rasterio
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, LSTM, Dense, Flatten, TimeDistributed, Concatenate
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy("mixed_float16")

In [4]:
# Build training data from GOES .nc files

goes_dir = "/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001"
smoke_output_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke.nc"
temp_output_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp.nc"

smoke_variables = [
    "MVFR_Fog_Prob",
    "IFR_Fog_Prob",
    "LIFR_Fog_Prob",
    "Fog_Depth",
]
temp_variables = [
    "Sfc_Temp_Bias",
]

def extract_timestamp_from_filename(filename):

    try:
        timestamp = filename.split('_s')[1][:12]
        return timestamp
    except IndexError:
        print(f"Error extracting timestamp from {filename}")
        return None

def filter_first_file_per_hour(nc_files):

    hourly_files = {}
    for nc_file in nc_files:
        timestamp = extract_timestamp_from_filename(nc_file)
        if not timestamp:
            continue
        hour = timestamp[8:10]
        if hour not in hourly_files:
            hourly_files[hour] = nc_file
    return list(hourly_files.values())

def process_goes_files(file_list, variables):

    datasets = []
    for file in file_list:
        try:
            with xr.open_dataset(file) as ds:
                selected_vars = {var: ds[var].load() for var in variables if var in ds.variables}
                if selected_vars:
                    datasets.append(xr.Dataset(selected_vars))
                else:
                    print(f"Warning: No matching variables found in {file}")
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    if not datasets:
        raise ValueError("No datasets were created. Please check the file list and variables.")

    combined_dataset = xr.concat(datasets, dim="time")
    return combined_dataset

def main():

    all_files = sorted([f for f in os.listdir(goes_dir) if f.endswith(".nc")])

    selected_files = filter_first_file_per_hour(all_files)
    selected_files = [os.path.join(goes_dir, f) for f in selected_files]
    print(f"GOES files to be used: {selected_files}")

    smoke_data = process_goes_files(selected_files, smoke_variables)
    smoke_data.to_netcdf(smoke_output_file)
    print(f"Smoke-related data has been saved to: {smoke_output_file}")

    temp_data = process_goes_files(selected_files, temp_variables)
    temp_data.to_netcdf(temp_output_file)
    print(f"Temperature-related data has been saved to: {temp_output_file}")

if __name__ == "__main__":
    main()

GOES files to be used: ['/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010001179_e202406010003552_c202406010005019.nc', '/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010101179_e202406010103552_c202406010104599.nc', '/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010201180_e202406010203553_c202406010204482.nc', '/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010301180_e202406010303553_c202406010304596.nc', '/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010401180_e202406010403553_c202406010404397.nc', '/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010501180_e202406010503553_c202406010504465.nc', '/content/drive/My Drive/WildFire/DATA/NOAA_GOES_R/001/ABI-L2-GFLSC-M6_v3r1_g18_s202406010601180_e202406010603553_c202406010605063.nc', '/content/drive/My Drive

In [21]:
def inspect_nc_file(nc_file):

    with xr.open_dataset(nc_file) as ds:
        print(f"Inspecting file: {nc_file}")
        print("\nVariables and Dimensions:")
        for var in ds.variables:
            print(f"{var}: {ds[var].dims} {ds[var].shape}")
        print("\n")

inspect_nc_file(smoke_output_file)
inspect_nc_file(temp_output_file)

def inspect_lat_lon_ranges(nc_file):
    with xr.open_dataset(nc_file) as ds:
        lat = ds['Latitude'].values
        lon = ds['Longitude'].values
        lat_min, lat_max = lat.min(), lat.max()
        lon_min, lon_max = lon.min(), lon.max()

    print(f"File: {nc_file}")
    print(f"Latitude range: {lat_min} to {lat_max}")
    print(f"Longitude range: {lon_min} to {lon_max}\n")

inspect_lat_lon_ranges(smoke_output_file)
inspect_lat_lon_ranges(temp_output_file)

Inspecting file: /content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke.nc

Variables and Dimensions:
Latitude: ('Rows', 'Columns') (1500, 2500)
Longitude: ('Rows', 'Columns') (1500, 2500)
MVFR_Fog_Prob: ('time', 'Rows', 'Columns') (24, 1500, 2500)
IFR_Fog_Prob: ('time', 'Rows', 'Columns') (24, 1500, 2500)
LIFR_Fog_Prob: ('time', 'Rows', 'Columns') (24, 1500, 2500)
Fog_Depth: ('time', 'Rows', 'Columns') (24, 1500, 2500)


Inspecting file: /content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp.nc

Variables and Dimensions:
Latitude: ('Rows', 'Columns') (1500, 2500)
Longitude: ('Rows', 'Columns') (1500, 2500)
Sfc_Temp_Bias: ('time', 'Rows', 'Columns') (24, 1500, 2500)


File: /content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke.nc
Latitude range: 14.571340560913086 to 53.500064849853516
Longitude range: -179.999267578125 to 179.99940490722656

File: /content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp.nc
Latitude range: 14.57134056091

In [23]:
def get_index_from_lat_lon(ds, lat_range, lon_range):

    lat = ds['Latitude'].values
    lon = ds['Longitude'].values

    row_start = np.argmin(np.abs(lat[:, 0] - lat_range[1]))
    row_end = np.argmin(np.abs(lat[:, 0] - lat_range[0]))

    col_start = np.argmin(np.abs(lon[0, :] - lon_range[0]))
    col_end = np.argmin(np.abs(lon[0, :] - lon_range[1]))

    return slice(row_start, row_end + 1), slice(col_start, col_end + 1)


def subset_nc_file(nc_file, lat_range, lon_range):

    with xr.open_dataset(nc_file) as ds:
        row_slice, col_slice = get_index_from_lat_lon(ds, lat_range, lon_range)
        print(f"Row slice: {row_slice.start} to {row_slice.stop - 1}")
        print(f"Column slice: {col_slice.start} to {col_slice.stop - 1}")

        ds_subset = ds.isel(Rows=row_slice, Columns=col_slice)
        return ds_subset

lat_range = [49.5, 52.5]
lon_range = [-116.0, -112.0]

smoke_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke.nc"
temp_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp.nc"

smoke_data_subset = subset_nc_file(smoke_file, lat_range, lon_range)
print(f"Subsetted smoke data dimensions:\n{smoke_data_subset.dims}\n")

temp_data_subset = subset_nc_file(temp_file, lat_range, lon_range)
print(f"Subsetted temperature data dimensions:\n{temp_data_subset.dims}\n")

output_smoke_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke_subset.nc"
output_temp_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp_subset.nc"

smoke_data_subset.to_netcdf(output_smoke_file)
print(f"Subsetted smoke data saved to: {output_smoke_file}")

temp_data_subset.to_netcdf(output_temp_file)
print(f"Subsetted temperature data saved to: {output_temp_file}")

Row slice: 22 to 94
Column slice: 1909 to 2021
Subsetted smoke data dimensions:
FrozenMappingWarningOnValuesAccess({'Rows': 73, 'Columns': 113, 'time': 24})

Row slice: 22 to 94
Column slice: 1909 to 2021
Subsetted temperature data dimensions:
FrozenMappingWarningOnValuesAccess({'Rows': 73, 'Columns': 113, 'time': 24})

Subsetted smoke data saved to: /content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke_subset.nc
Subsetted temperature data saved to: /content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp_subset.nc


In [24]:
def load_weather_data(nc_files, time_steps):
    weather_features = []
    for file in nc_files:
        with xr.open_dataset(file) as ds:
            feature = ds.to_array().values[:time_steps]
            weather_features.append(feature)
    return np.stack(weather_features, axis=-1)


def load_terrain_data(terrain_file, downscale_factor=4):
    with rasterio.open(terrain_file) as src:
        terrain = src.read(
            1, out_shape=(
                int(src.height // downscale_factor),
                int(src.width // downscale_factor)
            )
        )
    return terrain / np.max(terrain)

def load_smoke_and_temp(smoke_file, temp_file):

    with xr.open_dataset(smoke_file) as smoke_ds:
        smoke_data = smoke_ds.to_array().values

    with xr.open_dataset(temp_file) as temp_ds:
        temp_data = temp_ds.to_array().values

    time_ids = np.arange(1967184, 1967208)

    return smoke_data, temp_data, time_ids

def build_model(input_shape_smoke, input_shape_weather, input_shape_terrain):

    # Input for smoke data (CNN)
    smoke_input = Input(shape=input_shape_smoke, name="Smoke_Input")
    x = TimeDistributed(Conv2D(4, (3, 3), activation="relu", padding="same"))(smoke_input)
    x = TimeDistributed(Conv2D(8, (3, 3), activation="relu", padding="same"))(x)
    x = TimeDistributed(Flatten())(x)  # Shape: (time_steps, flattened_features)

    # Input for weather data (LSTM)
    weather_input = Input(shape=input_shape_weather, name="Weather_Input")
    w = LSTM(16, return_sequences=True)(weather_input)
    w = LSTM(16)(w)

    # Input for terrain data (Dense)
    terrain_input = Input(shape=input_shape_terrain, name="Terrain_Input")
    t = Dense(64, activation="relu")(terrain_input)  # Shape: (64,)

    # Combine all inputs
    combined = Concatenate()([Flatten()(x), w, t])  # Shape: (combined_features,)

    # Output layer
    output_size = input_shape_smoke[1] * input_shape_smoke[2]  # Height * Width
    out = Dense(output_size, activation="sigmoid")(combined)  # Flattened output
    out = tf.keras.layers.Reshape(target_shape=input_shape_smoke[1:])(out)  # Reshape to (height, width, channels)

    # Define model
    model = Model(inputs=[smoke_input, weather_input, terrain_input], outputs=out)

    return model

In [27]:
# File Directories
nc_files = ["/content/drive/My Drive/WildFire/DATA/NOAA_Climate/vwnd.2024.nc",
            "/content/drive/My Drive/WildFire/DATA/NOAA_Climate/air.2024.nc",
            "/content/drive/My Drive/WildFire/DATA/NOAA_Climate/hgt.2024.nc",
            "/content/drive/My Drive/WildFire/DATA/NOAA_Climate/omega.2024.nc",
            "/content/drive/My Drive/WildFire/DATA/NOAA_Climate/rhum.2024.nc",
            "/content/drive/My Drive/WildFire/DATA/NOAA_Climate/uwnd.2024.nc"]
terrain_file = "/content/drive/My Drive/WildFire/DATA/Terrain/terrain_broad_calgary.tiff"
smoke_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_smoke_subset.nc"
temp_file = "/content/drive/My Drive/WildFire/DATA/Processed/GOES_20240601_temp_subset.nc"

# Processing files

start_time = "2024-06-01T00:00:00"
end_time = "2024-06-01T23:00:00"

weather_data = load_weather_data(nc_files, start_time, end_time)
terrain_data = load_terrain_data(terrain_file)
smoke_data, temp_data, time_ids = load_smoke_and_temp(smoke_file, temp_file)

# Define input datashape
time_steps = smoke_data.shape[0]
input_shape_smoke = (time_steps, smoke_data.shape[1], smoke_data.shape[2], 1)
input_shape_weather = (time_steps, weather_data.shape[-1])
input_shape_terrain = (terrain_data.size,)

# Train model
model = build_model(input_shape_smoke, input_shape_weather, input_shape_terrain)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

x_smoke = smoke_data[:, :-1, :, :, np.newaxis]  # Cut off the last time_step to use the data as input
y_smoke = smoke_data[:, 1:, :, :, np.newaxis]   # Cut off the first time_step to use the data as output

x_weather = weather_data[:-1]
x_terrain = terrain_data

print(f"x_smoke shape: {x_smoke.shape}")
print(f"y_smoke shape: {y_smoke.shape}")
print(f"x_weather shape: {x_weather.shape}")
print(f"x_terrain shape: {x_terrain.shape}")

history = model.fit(
    x=[x_smoke, x_weather, x_terrain],
    y=y_smoke,
    epochs=10,
    batch_size=4,
    validation_split=0.2,
    shuffle=True,
)

print("Training complete.")
print("Final training loss:", history.history["loss"][-1])
print("Final validation loss:", history.history["val_loss"][-1])

x_smoke shape: (4, 23, 73, 113, 1)
y_smoke shape: (4, 23, 73, 113, 1)
x_weather shape: (0, 4, 17, 73, 144, 6)
x_terrain shape: (180, 240)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 3, 0, 3
'y' sizes: 3


In [ ]:
# A hint on how to use the predict model

current_smoke = smoke_data[-1:]
current_weather = weather_data[-1:]
current_terrain = terrain_data

predicted_smoke = model.predict([current_smoke, current_weather, current_terrain])